In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
=================================================================
Aktives und Passives Lernen für Dachmaterial-Klassifikation
=================================================================
Professionelles Skript für Dachmaterial-Experimente mit fairem Vergleich
zwischen aktivem und passivem Lernen.

Basiert auf dem MNIST Active Learning Framework, angepasst für
tabellarische Daten mit kategorischen und numerischen Features.

Dataset: umrisse_with_all_data_and_shape_and_patch_and_normal.csv
Zielvariable: mat_qgis (11 Klassen von Dachmaterialien)

Features:
- area: Numerisch (Fläche)
- area_type: Kategorisch
- Shape: Kategorisch
- ezg: Kategorisch

Classifiers:
- TabularNN (Neuronales Netz für tabellarische Daten)
- SVM, RandomForest, LogisticRegression, NaiveBayes

Strategien:
- least_confidence
- margin
- entropy
- information_density

Version: 1.0 - Dachmaterial-Version
"""

import time
import logging
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from functools import partial
from itertools import combinations
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
import os

# -------------------------------------------------------------------------------
# Reproduzierbarkeit
# -------------------------------------------------------------------------------
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# -------------------------------------------------------------------------------
# Logging konfigurieren
# -------------------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%H:%M:%S"
)
logger = logging.getLogger(__name__)

# -------------------------------------------------------------------------------
# 1) Daten laden und vorbereiten
# -------------------------------------------------------------------------------
def load_dachmaterial_data(filepath, target_col='mat_qgis', test_size=0.2, 
                          val_size=0.2, random_state=SEED, max_train=10000):
    """
    Lädt den Dachmaterial-Datensatz und bereitet ihn für ML vor.
    
    Args:
        filepath: Pfad zur CSV-Datei
        target_col: Zielvariable (Standard: 'mat_qgis')
        test_size: Anteil der Testdaten
        val_size: Anteil der Validierungsdaten vom Trainingsset
        random_state: Random Seed
        max_train: Maximale Anzahl Trainingsbeispiele
    
    Returns:
        Preprocessed data splits und zusätzliche Informationen
    """
    # Daten laden
    df = pd.read_csv(filepath)
    logger.info(f"Datensatz geladen: {len(df)} Zeilen, {len(df.columns)} Spalten")
    
    # Klassen-Verteilung anzeigen
    class_dist = df[target_col].value_counts()
    logger.info(f"Klassen-Verteilung:\n{class_dist}")
    
    # Feature-Spalten definieren
    feature_cols = ['area', 'area_type', 'Shape', 'ezg']
    
    # Überprüfe ob alle Features vorhanden sind
    missing_cols = [col for col in feature_cols if col not in df.columns]
    if missing_cols:
        logger.warning(f"Fehlende Spalten: {missing_cols}")
        feature_cols = [col for col in feature_cols if col in df.columns]
    
    # Datentypen analysieren
    logger.info("Analysiere Datentypen...")
    numeric_features = []
    categorical_features = []
    
    for col in feature_cols:
        if col in df.columns:
            dtype = df[col].dtype
            n_unique = df[col].nunique()
            n_missing = df[col].isna().sum()
            logger.info(f"  {col}: dtype={dtype}, unique={n_unique}, missing={n_missing}")
            
            # Bestimme ob Feature numerisch oder kategorisch ist
            if col == 'area':  # area ist definitiv numerisch
                numeric_features.append(col)
            elif 'type' in col.lower() or col in ['Shape', 'ezg']:
                categorical_features.append(col)
            elif df[col].dtype == 'object':
                categorical_features.append(col)
            elif df[col].nunique() < 20:
                categorical_features.append(col)
            else:
                numeric_features.append(col)
    
    logger.info(f"Numerische Features: {numeric_features}")
    logger.info(f"Kategoriale Features: {categorical_features}")
    
    # Nur Zeilen mit gültiger Zielvariable behalten
    df = df[df[target_col].notna()].copy()
    
    # Features und Target trennen
    X = df[feature_cols].copy()
    y = df[target_col].copy()
    
    # Label Encoding für Target
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    n_classes = len(label_encoder.classes_)
    logger.info(f"Anzahl Klassen: {n_classes}")
    logger.info(f"Klassen: {list(label_encoder.classes_)}")
    
    # Preprocessing Pipeline erstellen
    # Numerische Features: Imputing + Scaling
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    
    # Kategorische Features: Imputing + One-Hot Encoding
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
    ])
    
    # ColumnTransformer kombiniert beide
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ])
    
    # Stratified Split für ausgeglichene Klassen-Verteilung
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y_encoded, test_size=test_size, random_state=random_state, 
        stratify=y_encoded
    )
    
    # Subsampling falls gewünscht
    if len(X_temp) > max_train:
        # Stratified sampling für Subsampling
        _, X_temp, _, y_temp = train_test_split(
            X_temp, y_temp, test_size=max_train/len(X_temp), 
            random_state=random_state, stratify=y_temp
        )
    
    # Train/Val Split
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=val_size, random_state=random_state,
        stratify=y_temp
    )
    
    # Preprocessor fitten und transformieren
    X_train_processed = preprocessor.fit_transform(X_train).astype(np.float32)
    X_val_processed = preprocessor.transform(X_val).astype(np.float32)
    X_test_processed = preprocessor.transform(X_test).astype(np.float32)
    
    # Feature-Namen für spätere Verwendung extrahieren
    feature_names = []
    if hasattr(preprocessor, 'get_feature_names_out'):
        feature_names = list(preprocessor.get_feature_names_out())
    else:
        # Fallback für ältere sklearn Versionen
        for name, transformer, features in preprocessor.transformers_:
            if name == 'num':
                feature_names.extend(features)
            elif name == 'cat' and hasattr(transformer.named_steps['onehot'], 'get_feature_names_out'):
                cat_features = transformer.named_steps['onehot'].get_feature_names_out(features)
                feature_names.extend(cat_features)
    
    logger.info(f"Daten vorbereitet: Train={len(X_train)}, Val={len(X_val)}, Test={len(X_test)}")
    logger.info(f"Feature-Dimension nach Preprocessing: {X_train_processed.shape[1]}")
    
    return (X_train_processed, y_train, 
            X_val_processed, y_val,
            X_test_processed, y_test,
            preprocessor, label_encoder, feature_names, n_classes)

# -------------------------------------------------------------------------------
# 2) Neuronales Netz für tabellarische Daten
# -------------------------------------------------------------------------------
class TabularNN(nn.Module):
    """
    Neuronales Netz optimiert für tabellarische Daten mit kategorischen und
    numerischen Features. Verwendet Batch Normalization und Dropout.
    """
    def __init__(self, input_dim, n_classes, hidden_dims=[128, 64, 32], dropout_rate=0.3):
        super().__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        self.dropouts = nn.ModuleList()
        self.activation = nn.ReLU()
        
        # Input layer
        self.layers.append(nn.Linear(input_dim, hidden_dims[0]))
        self.batch_norms.append(nn.BatchNorm1d(hidden_dims[0]))
        self.dropouts.append(nn.Dropout(dropout_rate))
        
        # Hidden layers
        for i in range(len(hidden_dims) - 1):
            self.layers.append(nn.Linear(hidden_dims[i], hidden_dims[i+1]))
            self.batch_norms.append(nn.BatchNorm1d(hidden_dims[i+1]))
            self.dropouts.append(nn.Dropout(dropout_rate))
        
        # Output layer
        self.output_layer = nn.Linear(hidden_dims[-1], n_classes)
        
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)
        logger.info(f"TabularNN initialisiert auf {self.device}")

    def forward(self, x):
        for i, (layer, bn, dropout) in enumerate(zip(self.layers, self.batch_norms, self.dropouts)):
            x = layer(x)
            if x.shape[0] > 1:  # Batch normalization benötigt batch_size > 1
                x = bn(x)
            x = self.activation(x)
            x = dropout(x)
        return self.output_layer(x)

    def fit_nn(self, X_np, y_np, X_val=None, y_val=None, epochs=50, lr=1e-3, 
               batch_size=128, patience=10):
        """
        Trainiert das Neuronale Netz mit Early Stopping.
        """
        self.train()
        optimizer = optim.Adam(self.parameters(), lr=lr, weight_decay=1e-4)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)
        loss_fn = nn.CrossEntropyLoss()
        
        # Dataset und DataLoader
        dataset = TensorDataset(
            torch.from_numpy(X_np).float(),
            torch.from_numpy(y_np).long()
        )
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        
        # Early Stopping
        best_val_loss = float('inf')
        patience_counter = 0
        best_state = None
        
        for epoch in range(epochs):
            total_loss = 0
            for xb, yb in loader:
                xb, yb = xb.to(self.device), yb.to(self.device)
                optimizer.zero_grad()
                output = self(xb)
                loss = loss_fn(output, yb)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            
            # Validierung
            if X_val is not None and y_val is not None:
                self.eval()
                with torch.no_grad():
                    X_val_t = torch.from_numpy(X_val).float().to(self.device)
                    y_val_t = torch.from_numpy(y_val).long().to(self.device)
                    val_output = self(X_val_t)
                    val_loss = loss_fn(val_output, y_val_t).item()
                
                scheduler.step(val_loss)
                
                # Early Stopping Check
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_state = self.state_dict().copy()
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if patience_counter >= patience:
                        if best_state is not None:
                            self.load_state_dict(best_state)
                        break
                
                self.train()
                
        return self

    def predict_proba(self, X_np):
        """
        Gibt die Klassenwahrscheinlichkeiten zurück.
        """
        self.eval()
        with torch.no_grad():
            X_t = torch.from_numpy(X_np).float().to(self.device)
            logits = self(X_t)
            probs = torch.softmax(logits, dim=1)
        return probs.cpu().numpy()

# -------------------------------------------------------------------------------
# 3) Klassische Klassifikatoren
# -------------------------------------------------------------------------------
CLASSIFIERS = {
    'SVM': partial(SVC, probability=True, kernel='rbf', gamma='scale', random_state=SEED),
    'RandomForest': partial(RandomForestClassifier, n_estimators=100, max_depth=10, 
                           min_samples_split=5, n_jobs=-1, random_state=SEED),
    'LogisticRegression': partial(LogisticRegression, solver='lbfgs', max_iter=1000, 
                                 random_state=SEED, n_jobs=-1),
    'NaiveBayes': partial(GaussianNB),
    'TabularNN': None  # Wird separat behandelt
}

def train_and_predict(clf_key, X_tr, y_tr, X_te, X_val=None, y_val=None, n_classes=None):
    """
    Trainiert das gewünschte Modell und gibt Predictions + Modell zurück.
    """
    if clf_key == 'TabularNN':
        if n_classes is None:
            n_classes = len(np.unique(y_tr))
        input_dim = X_tr.shape[1]
        nn = TabularNN(input_dim=input_dim, n_classes=n_classes)
        nn.fit_nn(X_tr, y_tr, X_val, y_val, epochs=50, patience=10)
        preds = np.argmax(nn.predict_proba(X_te), axis=1)
        return preds, nn
    else:
        model = CLASSIFIERS[clf_key]()
        model.fit(X_tr, y_tr)
        preds = model.predict(X_te)
        return preds, model

# -------------------------------------------------------------------------------
# 4) Query-/Uncertainty-Funktionen (identisch mit MNIST)
# -------------------------------------------------------------------------------
def predict_proba(model, X):
    """
    Vereinheitlichte Probability-Abfrage für NN und Sklearn-Modelle.
    """
    if isinstance(model, TabularNN):
        return model.predict_proba(X)
    else:
        return model.predict_proba(X)

def least_confidence(model, X, k=1):
    """
    Wählt k Samples mit geringster "Confidence" (max. Klasse).
    """
    p = predict_proba(model, X)
    scores = np.max(p, axis=1)
    return np.argsort(scores)[:k]

def margin(model, X, k=1):
    """
    Wählt k Samples mit kleinstem Margin zwischen den Top-2 Klassen.
    """
    p = predict_proba(model, X)
    top2 = np.sort(p, axis=1)[:, -2:]
    margins = top2[:, 1] - top2[:, 0]
    return np.argsort(margins)[:k]

def entropy_uncertainty(model, X, k=1):
    """
    Wählt k Samples mit höchster Entropie aus.
    """
    p = predict_proba(model, X)
    ent = (-p * np.log(p + 1e-9)).sum(axis=1)
    return np.argsort(ent)[-k:]

def information_density(model, X, k=1, subsample_size=1000):
    """
    Mischt Entropie mit "Density" im Feature-Space.
    """
    p = predict_proba(model, X)
    ent = (-p * np.log(p + 1e-9)).sum(axis=1)

    # Für große Pools: Subsample für Density-Berechnung
    if len(X) > subsample_size:
        sub_idx = np.random.choice(len(X), subsample_size, replace=False)
        X_sub = X[sub_idx]
    else:
        X_sub = X
        sub_idx = np.arange(len(X))

    # Berechne Durchschnittsdistanz zu k nächsten Nachbarn
    k_neighbors = min(10, len(X_sub) - 1)
    densities = np.zeros(len(X))

    for i in range(len(X_sub)):
        dists = np.sqrt(((X_sub[i] - X_sub) ** 2).sum(axis=1))
        dists[i] = np.inf  # Exclude self
        nearest = np.sort(dists)[:k_neighbors]
        densities[sub_idx[i]] = 1.0 / (nearest.mean() + 1e-9)

    # Für nicht-subsample Punkte: Durchschnittliche Density
    if len(X) > subsample_size:
        mean_density = densities[sub_idx].mean()
        densities[densities == 0] = mean_density

    scores = ent * densities
    return np.argsort(scores)[-k:]

# -------------------------------------------------------------------------------
# 5) Active-Learning-Schleife (angepasst für Dachmaterial)
# -------------------------------------------------------------------------------
def active_learning(X_train, y_train, X_val, y_val, X_test, y_test,
                    strategy, clf_key, budget, runs, n_classes,
                    run_offset=0, batch_size=50):
    """
    Führt Active Learning mit gegebener Strategie, Klassifikator und Budget durch.
    
    WICHTIG: Der aktive Lerner hat Zugriff auf den GESAMTEN Trainingspool.
    """
    results = []
    n = len(X_train)
    n_label = int(budget * n)

    if n_label < batch_size:
        logger.warning(f"Budget zu klein (n_label={n_label} < batch_size={batch_size})")
        return results

    for run_i in range(runs):
        run_id = run_offset + run_i
        logger.info(f"[{clf_key}][{strategy}] Run {run_i+1}/{runs} — Budget {budget:.1%} ({n_label} samples)")

        # ================================
        # PASSIVES LERNEN (Baseline)
        # ================================
        idx_passive = np.random.choice(n, n_label, replace=False)

        t0 = time.time()
        y_pred_passive, _ = train_and_predict(
            clf_key,
            X_train[idx_passive],
            y_train[idx_passive],
            X_test,
            X_val, y_val,
            n_classes
        )
        pass_train_time = time.time() - t0

        # Ergebnisse (passiv)
        acc_passive = accuracy_score(y_test, y_pred_passive)
        f1_passive = f1_score(y_test, y_pred_passive, average='macro')

        results.append([
            "passiv",
            clf_key,
            strategy,
            budget,
            run_id,
            "final",
            n_label,
            acc_passive,
            f1_passive,
            pass_train_time,
            0.0  # diversity
        ])

        logger.info(f"  Passiv: {n_label} labels → Acc={acc_passive:.3f}, F1={f1_passive:.3f}")

        # ================================
        # AKTIVES LERNEN
        # ================================
        # Der Pool ist das GESAMTE Trainingsset
        is_labeled = np.zeros(n, dtype=bool)
        labeled_indices = []

        q_steps = int(np.ceil(n_label / batch_size))

        for q in range(q_steps):
            # Anzahl zu labelender Samples in diesem Schritt
            remaining_budget = n_label - len(labeled_indices)
            b_size_current = min(batch_size, remaining_budget)

            if b_size_current <= 0:
                break

            # Unlabeled pool indices
            unlabeled_mask = ~is_labeled
            unlabeled_indices = np.where(unlabeled_mask)[0]

            # Modell trainieren, falls schon gelabelte Daten vorliegen
            if len(labeled_indices) > 0:
                t0 = time.time()
                X_lab = X_train[labeled_indices]
                y_lab = y_train[labeled_indices]
                _, model = train_and_predict(clf_key, X_lab, y_lab, X_test, X_val, y_val, n_classes)
                train_time = time.time() - t0
            else:
                train_time = 0.0
                model = None

            # Auswahl via aktiver Strategie
            if model is None or len(labeled_indices) < 10:  # Bootstrap mit Random
                selected_pool_idx = np.random.choice(
                    len(unlabeled_indices),
                    size=min(b_size_current, len(unlabeled_indices)),
                    replace=False
                )
            else:
                # Aktive Auswahl aus dem unlabeled Pool
                X_unlabeled = X_train[unlabeled_indices]

                if strategy == 'least_confidence':
                    selected_pool_idx = least_confidence(model, X_unlabeled, k=b_size_current)
                elif strategy == 'margin':
                    selected_pool_idx = margin(model, X_unlabeled, k=b_size_current)
                elif strategy == 'entropy':
                    selected_pool_idx = entropy_uncertainty(model, X_unlabeled, k=b_size_current)
                elif strategy == 'information_density':
                    selected_pool_idx = information_density(model, X_unlabeled, k=b_size_current)
                else:
                    selected_pool_idx = entropy_uncertainty(model, X_unlabeled, k=b_size_current)

            # Konvertiere Pool-Indizes zu globalen Indizes
            selected_global_idx = unlabeled_indices[selected_pool_idx]

            # Diversität der ausgewählten Samples berechnen
            if len(selected_global_idx) > 1:
                feats = X_train[selected_global_idx]
                pairwise = list(combinations(feats, 2))
                diversity = np.mean([np.linalg.norm(a - b) for a, b in pairwise]) if pairwise else 0.0
            else:
                diversity = 0.0

            # Update labeled indices und mask
            labeled_indices.extend(selected_global_idx.tolist())
            is_labeled[selected_global_idx] = True

            # Test-Evaluation
            if len(labeled_indices) > 0:
                X_lab = X_train[labeled_indices]
                y_lab = y_train[labeled_indices]
                y_eval, _ = train_and_predict(clf_key, X_lab, y_lab, X_test, X_val, y_val, n_classes)
                acc = accuracy_score(y_test, y_eval)
                f1  = f1_score(y_test, y_eval, average='macro')
            else:
                acc = 0.0
                f1 = 0.0

            # Schritt-Ergebnis loggen
            results.append([
                "aktiv",
                clf_key,
                strategy,
                budget,
                run_id,
                q,
                len(labeled_indices),
                acc,
                f1,
                train_time,
                diversity
            ])

            # Jeden 5. Schritt loggen
            if q % 5 == 0:
                logger.info(f"  Aktiv [{strategy}] Schritt {q+1}/{q_steps}: "
                          f"{len(labeled_indices)} labels → Acc={acc:.3f}, F1={f1:.3f}")

    return results

# -------------------------------------------------------------------------------
# 6) Evaluation Utilities
# -------------------------------------------------------------------------------
def print_summary_statistics(df, label_encoder=None):
    """
    Gibt eine übersichtliche Zusammenfassung der Ergebnisse aus.
    """
    # Finale Ergebnisse (letzte Iteration jedes Runs)
    final_results = df.groupby(['lernmodus', 'klassifizierer', 'strategie', 'budget', 'run_id']).last().reset_index()

    # Aggregierte Statistiken
    summary = final_results.groupby(['lernmodus', 'klassifizierer', 'strategie', 'budget']).agg({
        'accuracy': ['mean', 'std'],
        'f1_macro': ['mean', 'std'],
        'train_time': ['mean', 'std']
    }).round(4)

    print("\n" + "="*80)
    print("ZUSAMMENFASSUNG DER ERGEBNISSE")
    print("="*80)
    print(summary)

    # Vergleich Aktiv vs Passiv pro Budget
    for budget in sorted(df['budget'].unique()):
        print(f"\n{'='*60}")
        print(f"BUDGET: {budget:.1%}")
        print(f"{'='*60}")

        budget_data = final_results[final_results['budget'] == budget]

        for clf in sorted(df['klassifizierer'].unique()):
            clf_data = budget_data[budget_data['klassifizierer'] == clf]
            passive_clf = clf_data[clf_data['lernmodus'] == 'passiv']['accuracy']

            if passive_clf.empty:
                continue

            passive_mean = passive_clf.mean()
            passive_std = passive_clf.std()

            print(f"\n{clf}:")
            print(f"  Passiv: {passive_mean:.3f} ± {passive_std:.3f}")

            improvements = []
            for strategy in sorted(df['strategie'].unique()):
                active_data = clf_data[(clf_data['lernmodus'] == 'aktiv') &
                                      (clf_data['strategie'] == strategy)]
                if not active_data.empty:
                    active_acc = active_data['accuracy']
                    active_mean = active_acc.mean()
                    active_std = active_acc.std()
                    improvement = (active_mean - passive_mean) * 100
                    improvements.append((strategy, active_mean, active_std, improvement))
                    print(f"  Aktiv ({strategy:20}): {active_mean:.3f} ± {active_std:.3f} ({improvement:+.1f}%)")

            # Beste Strategie markieren
            if improvements:
                best_strategy = max(improvements, key=lambda x: x[1])
                print(f"  → Beste Strategie: {best_strategy[0]} mit {best_strategy[3]:+.1f}% Verbesserung")

def create_visualizations(df, label_encoder=None, output_dir='plots_dachmaterial'):
    """
    Erstellt Visualisierungen der Ergebnisse.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Style setzen
    plt.style.use('seaborn-v0_8-whitegrid')
    colors = sns.color_palette("husl", n_colors=len(df['strategie'].unique()))
    
    # 1. Learning Curves pro Budget
    for budget in sorted(df['budget'].unique()):
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.flatten()
        
        budget_data = df[df['budget'] == budget]
        
        for idx, clf in enumerate(sorted(df['klassifizierer'].unique())):
            if idx >= len(axes):
                break
                
            ax = axes[idx]
            clf_data = budget_data[budget_data['klassifizierer'] == clf]
            
            # Passive baseline
            passive_data = clf_data[clf_data['lernmodus'] == 'passiv']
            if not passive_data.empty:
                passive_acc = passive_data['accuracy'].mean()
                ax.axhline(y=passive_acc, color='black', linestyle='--', 
                          label='Passiv', alpha=0.7, linewidth=2)
            
            # Active learning curves
            for i, strategy in enumerate(sorted(df['strategie'].unique())):
                strategy_data = clf_data[(clf_data['lernmodus'] == 'aktiv') &
                                       (clf_data['strategie'] == strategy)]
                
                if not strategy_data.empty:
                    grouped = strategy_data.groupby('anz_label')['accuracy'].agg(['mean', 'std'])
                    ax.plot(grouped.index, grouped['mean'], 
                           label=strategy, marker='o', color=colors[i], linewidth=2)
                    ax.fill_between(grouped.index, 
                                  grouped['mean'] - grouped['std'],
                                  grouped['mean'] + grouped['std'],
                                  alpha=0.3, color=colors[i])
            
            ax.set_xlabel('Anzahl Labels', fontsize=12)
            ax.set_ylabel('Accuracy', fontsize=12)
            ax.set_title(f'{clf}', fontsize=14, fontweight='bold')
            ax.legend(loc='best', fontsize=10)
            ax.grid(True, alpha=0.3)
            ax.set_ylim(0, 1)
        
        # Remove empty subplots
        for idx in range(len(df['klassifizierer'].unique()), len(axes)):
            fig.delaxes(axes[idx])
        
        plt.suptitle(f'Learning Curves - Budget {budget:.1%}', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig(f'{output_dir}/learning_curves_budget_{int(budget*100)}.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
    
    # 2. Performance Heatmap
    final_results = df.groupby(['lernmodus', 'klassifizierer', 'strategie', 'budget', 'run_id']).last().reset_index()
    active_results = final_results[final_results['lernmodus'] == 'aktiv']
    
    # Erstelle Pivot-Tabelle für Heatmap
    pivot_data = active_results.groupby(['klassifizierer', 'strategie', 'budget'])['accuracy'].mean().unstack(level=[1, 2])
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(pivot_data, annot=True, fmt='.3f', cmap='YlOrRd', 
                cbar_kws={'label': 'Accuracy'}, ax=ax)
    ax.set_title('Performance Heatmap: Klassifizierer × Strategie × Budget', fontsize=16)
    ax.set_xlabel('Strategie × Budget', fontsize=12)
    ax.set_ylabel('Klassifizierer', fontsize=12)
    plt.tight_layout()
    plt.savefig(f'{output_dir}/performance_heatmap.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # 3. Confusion Matrix für besten Klassifikator
    best_config = active_results.groupby(['klassifizierer', 'strategie', 'budget'])['accuracy'].mean().idxmax()
    logger.info(f"Beste Konfiguration: {best_config}")
    
    logger.info(f"Visualisierungen gespeichert in '{output_dir}/'")

# -------------------------------------------------------------------------------
# Hauptprogramm
# -------------------------------------------------------------------------------
def main():
    """
    Haupt-Einstiegspunkt:
    1) Dachmaterial-Daten laden
    2) Budget- und Strategie-Settings definieren
    3) Experiment-Schleife
    4) Ergebnisse speichern und zusammenfassen
    """
    # Konfiguration
    data_file = "umrisse_with_all_data_and_shape_and_patch_and_normal.csv"
    target_column = "mat_qgis"
    
    # 1) Daten laden
    logger.info("="*80)
    logger.info("ACTIVE LEARNING FÜR DACHMATERIAL-KLASSIFIKATION")
    logger.info("="*80)
    
    try:
        (X_train, y_train, X_val, y_val, X_test, y_test,
         preprocessor, label_encoder, feature_names, n_classes) = load_dachmaterial_data(
            filepath=data_file,
            target_col=target_column,
            test_size=0.2,
            val_size=0.2,
            random_state=SEED,
            max_train=10000  # Kann erhöht werden für umfangreichere Experimente
        )
    except FileNotFoundError:
        logger.error(f"Datei '{data_file}' nicht gefunden!")
        return
    except Exception as e:
        logger.error(f"Fehler beim Laden der Daten: {e}")
        return

    # 2) Experiment-Konfiguration
    budgets = [0.3, 0.6, 0.9]  # 30%, 60%, 90% des Trainingssets
    strategies = ["least_confidence", "margin", "entropy", "information_density"]
    clf_keys = ["TabularNN", "SVM", "RandomForest", "LogisticRegression", "NaiveBayes"]
    runs = 5  # Anzahl Wiederholungen pro Setting

    logger.info(f"\nExperiment-Konfiguration:")
    logger.info(f"Budgets: {budgets}")
    logger.info(f"Strategien: {strategies}")
    logger.info(f"Klassifikatoren: {clf_keys}")
    logger.info(f"Runs pro Konfiguration: {runs}")

    # 3) Experiment-Schleife
    all_results = []
    total_experiments = len(strategies) * len(clf_keys) * len(budgets)
    exp_count = 0

    start_time = time.time()

    for budget in budgets:
        for clf_key in clf_keys:
            for strategy in strategies:
                exp_count += 1
                print(f"\n[{exp_count}/{total_experiments}] "
                      f"Budget={budget:.1%}, Classifier={clf_key}, Strategy={strategy}")

                res = active_learning(
                    X_train, y_train,
                    X_val,   y_val,
                    X_test,  y_test,
                    strategy=strategy,
                    clf_key=clf_key,
                    budget=budget,
                    runs=runs,
                    n_classes=n_classes,
                    batch_size=50
                )
                all_results.extend(res)

    # 4) DataFrame bauen und als CSV ablegen
    df = pd.DataFrame(
        all_results,
        columns=[
            "lernmodus",      # "passiv" oder "aktiv"
            "klassifizierer", # "TabularNN", "SVM", ...
            "strategie",      # "least_confidence", ...
            "budget",         # 0.3, 0.6, 0.9
            "run_id",
            "zyklus",
            "anz_label",
            "accuracy",
            "f1_macro",
            "train_time",
            "diversity"
        ]
    )

    # Ergebnisse speichern
    output_file = "ergebnisse_dachmaterial_classification.csv"
    df.to_csv(output_file, index=False)
    logger.info(f"\nErgebnisse in '{output_file}' gespeichert.")

    # Zusammenfassung ausgeben
    print_summary_statistics(df, label_encoder)
    
    # Visualisierungen erstellen
    create_visualizations(df, label_encoder)
    
    # Gesamtzeit
    total_time = time.time() - start_time
    logger.info(f"\nGesamte Laufzeit: {total_time/60:.1f} Minuten")
    
    # Top-Performer identifizieren
    print("\n" + "="*80)
    print("TOP 5 KONFIGURATIONEN (nach durchschnittlicher Accuracy)")
    print("="*80)
    
    final_results = df.groupby(['lernmodus', 'klassifizierer', 'strategie', 'budget', 'run_id']).last().reset_index()
    active_results = final_results[final_results['lernmodus'] == 'aktiv']
    
    top_configs = active_results.groupby(['klassifizierer', 'strategie', 'budget']).agg({
        'accuracy': 'mean',
        'f1_macro': 'mean',
        'train_time': 'mean'
    }).round(4).sort_values('accuracy', ascending=False).head(5)
    
    print(top_configs)


if __name__ == "__main__":
    main()